In [1]:
import os, sys, time, resource, re, gc, shutil
import operator
from multiprocess import Pool
from functools import partial
from urllib.parse import urlparse, parse_qsl
from django.db.models import Count
import matplotlib


mp_pdf = False
if mp_pdf:
    extension = "pdf"
    import matplotlib
    matplotlib.use('pgf')
    pgf_with_latex = {
        "text.usetex": True,            # use LaTeX to write all text
        "pgf.rcfonts": False,           # Ignore Matplotlibrc
        "text.latex.unicode": True,
        "pgf.preamble": [
            #r"\usepackage[utf8x]{inputenc}",
            r"\usepackage{xcolor}"
        ],
        "pgf.texsystem" : "xelatex",
        "figure.figsize": [12,7]
    }
    matplotlib.rcParams.update(pgf_with_latex)
else:
    extension = "png"

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import django
from adjustText import adjust_text

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
from django.db.models import F, Sum, Count
from matplotlib import gridspec

from utils.utils import *

from run_id import run_id

run_id = 1771

runstat = RunStats.objects.get(pk=run_id)
topics = DynamicTopic.objects.filter(run_id=run_id)

docs = Doc.objects.filter(
    query=runstat.query,
    content__iregex="\w"
)



In [ ]:
years = list(range(1990,2015))
years

s_solutions = []
ip_solutions = []

r = "(?:\W|^)solution|political action|policy"

for y in years:
    ydocs = docs.filter(PY=y)
    sdocs = ydocs.filter(content__iregex=r)
    s_count = sdocs.count()
    s_solutions.append(s_count/ydocs.count())
    ip_docs = ydocs.filter(ipccref__isnull=False)
    ip_solution_docs = ip_docs.filter(content__iregex=r)
    try:
        ip_solutions.append(ip_solution_docs.count()/ip_docs.count())
    except:
        ip_solutions.append(0)
    #ipdocs = ydocs.filter()
    

In [ ]:
fig = plt.figure()

plt.plot(years,s_solutions)
plt.plot(years, ip_solutions)

fig.patch.set_facecolor('#f0f0f0')    
#plt.tight_layout()

plt.savefig(
    f'../plots/solutions.{extension}',
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
)
plt.show()

In [ ]:
if runstat.method=="DT":
    dts = DocTopic.objects.filter(
        run_id=run_id,
        doc__PY__lt=2014,
        score__gt=runstat.dt_threshold,
        topic__topicdtopic__dynamictopic__run_id=run_id,
        topic__topicdtopic__score__gt=runstat.dt_threshold
    ).values('topic__topicdtopic__dynamictopic__id')

    dts = dts.annotate(
        solution = models.Sum(
            models.Case(
                models.When(
                    doc__content__iregex=r,
                    then=F('score')*F('topic__topicdtopic__score')
                ),
                default=0,
                output_field=models.FloatField()
            )
        )
    )

    df = pd.DataFrame.from_dict(list(dts.values(
        'topic__topicdtopic__dynamictopic__title',
        'solution',
        'topic__topicdtopic__dynamictopic__score'
    ))).rename(columns={
        'topic__topicdtopic__dynamictopic__title':'topic__title',
        'topic__topicdtopic__dynamictopic__score': 'score'
    })
else:
    dts = DocTopic.objects.filter(
        run_id=run_id,
        doc__PY__lt=2014,
        score__gt=runstat.dt_threshold,
    ).values('topic__title')

    dts = dts.annotate(
        solution = models.Sum(
            models.Case(
                models.When(
                    doc__content__iregex=r,
                    then=F('score')
                ),
                default=0,
                output_field=models.FloatField()
            )
        )
    )

    df = pd.DataFrame.from_dict(list(dts.values(
        'topic__title',
        'solution',
        'score'
    )))
    
df.head()
                                     
                                    


In [2]:
topics = Topic.objects.filter(
    run_id=run_id,
    #primary_wg=3
).values('doctopic__doc__wc__oecd').annotate(
    dis_share = Sum('doctopic__score')
)

topic_dis_df = pd.DataFrame.from_dict(list(topics.values(
    'title','doctopic__doc__wc__oecd','dis_share','primary_wg'
)))

print(topic_dis_df.shape)

topic_dis_df.head()

(700, 4)


,dis_share,doctopic__doc__wc__oecd,primary_wg,title
0,40.160252,Agricultural Sciences,2,"{adaptation, vulnerability, farmer}"
1,13.444909,Agricultural Sciences,2,"{adsorption, capacity, adsorbent}"
2,5.938442,Agricultural Sciences,1,"{aerosol, forcing, radiative}"
3,15.398295,Agricultural Sciences,1,"{arctic, warm, tundra}"
4,68.393232,Agricultural Sciences,2,"{area, region, distribution}"


In [5]:
topic_dis_wide = topic_dis_df.pivot_table(
    index=["title","primary_wg"],columns="doctopic__doc__wc__oecd",values="dis_share"
).reset_index()
topic_dis_wide.head()

doctopic__doc__wc__oecd,title,primary_wg,Agricultural Sciences,Engineering and Technology,Humanities,Medical and Health Sciences,Natural Sciences,Social Sciences
0,"{adaptation, vulnerability, farmer}",2,40.160252,49.852006,2.723019,7.361568,224.354252,97.406139
1,"{adsorption, capacity, adsorbent}",2,13.444909,117.865230,0.589684,1.479196,173.869152,8.536607
2,"{aerosol, forcing, radiative}",1,5.938442,41.617363,0.533181,1.017193,199.348419,5.158763
3,"{arctic, warm, tundra}",1,15.398295,29.900207,1.713840,3.199851,319.806277,16.604214
4,"{area, region, distribution}",2,68.393232,114.825359,3.263985,10.517513,476.263762,46.541184


In [ ]:

fig, ax = plt.subplots(figsize=(12,12))

cols = [
    "primary_wg",
    "Agricultural Sciences", 
    "Engineering and Technology",
    "Natural Sciences", 
    "Social Sciences"
]

rad_viz = pd.plotting.radviz(
    topic_dis_wide[cols], 
    'primary_wg', 
    ax=ax,
    color=["#66c2a5","#fc8d62","#8da0cb"],
    edgecolors="grey",
    linewidths=0.1,
    s = 20
) 

plt.show()

In [8]:
topic_dis_df.to_csv('../tables/topic_discipline.csv',index=False)